# Addressing early questions

>  the ones I noticed are: DAY2 I am 95% sure did not collect resting BOLD, and I think the same is probably true for FNDM1 (which was originally called just FNDM).

In [1]:
import flywheel

In [2]:
client=flywheel.Client()

In [3]:
proj = client.lookup('bbl/Reward2018')

/cbica/projects/wolf_satterthwaite_reward/miniconda3/envs/reward/lib/python3.8/site-packages/flywheel/flywheel.py:6300: UserWarning: Client version 15.8.0 does not match server version 14.6.4. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [4]:
proj.id

'5c1a61e69011bd0011368884'

Did DAY2 collect resting state bold data?

In [5]:
sessions = client.get_project_sessions(proj.id)

In [6]:
len(sessions)

509

In [7]:
day2 = [x for x in sessions if 'day2' in x.label.lower()]

In [8]:
acquisitions = []

for d in day2[:10]:
    
    acqs = d.acquisitions()
    print([x.label for x in acqs])

['bbl1_faceB0_178', 'localizer_32channel', 'MPRAGE_TI1100_ipat2', 'bbl1_cardA0_178', 'MoCoSeries', 'bbl1_faceA0_178', 'epi_singlerep', 'ep2d_se_pcasl_PHC_1200ms', 'bbl1_cardB0_178', 'ep2d_bold_MGH', 'B0map_onesizefitsall_v2', 'B0map_onesizefitsall_v2']
['bbl1_faceB0_178', 'MPRAGE_TI1100_ipat2', 'B0map_onesizefitsall_v2', 'MoCoSeries', 'bbl1_faceA0_178', 'bbl1_cardB0_178', 'bbl1_cardA0_178', 'ep2d_se_pcasl_PHC_1200ms', 'epi_singlerep', 'ep2d_bold_MGH', 'localizer_32channel', 'B0map_onesizefitsall_v2']
['ep2d_bold_MGH', 'B0map_onesizefitsall_v2', 'bbl1_faceA0_178', 'MPRAGE_TI1100_ipat2', 'bbl1_faceB0_178', 'epi_singlerep', 'bbl1_cardB0_178', 'MoCoSeries', 'bbl1_cardA0_178', 'B0map_onesizefitsall_v2', 'ep2d_se_pcasl_PHC_1200ms', 'localizer_32channel']
['bbl1_faceB0_178', 'B0map_onesizefitsall_v2', 'bbl1_cardA0_178', 'B0map_onesizefitsall_v2', 'MoCoSeries', 'epi_singlerep', 'bbl1_cardB0_178', 'ep2d_bold_MGH', 'bbl1_faceA0_178', 'localizer_32channel', 'ep2d_se_pcasl_PHC_1200ms', 'MPRAGE_TI1

In [9]:
def get_nested(dct, *keys):
    for key in keys:
        try:
            dct = dct[key]
        except (KeyError, TypeError):
            return None
    return dct

def process_acq(acq):
    
    files = acq.files
    
    names = {}
    
    for f in files:
        
        if f.type != 'nifti':
            
            continue

        bids_filename = get_nested(f, 'info', 'BIDS', 'Filename')

        if bids_filename is None:
            continue
        
        day_2_start = bids_filename.find('day2')
        end_ses = bids_filename[day_2_start:].find("_")
        bids_filename = bids_filename[day_2_start+end_ses+1:].replace(".nii.gz", "")
        
        if bids_filename not in names.keys():
            names[bids_filename] = 1
        else:
            names[bids_filename] += 1
        
    return names
    
    

In [10]:
process_acq(acqs[0])

{'phase2': 1, 'phase1': 1}

In [11]:
acquisitions = {}

for d in day2:
    acqs = d.acquisitions()

    for a in acqs:
                
        
        if a.label not in acquisitions.keys():
            
            names = process_acq(a)
            
            acquisitions[a.label] = names
            
        else:
            
            names = process_acq(a)

            for k, n in names.items():
                
                if k not in acquisitions[a.label].keys():
                    
                    acquisitions[a.label][k] = n 
                    
                else:
                    
                    acquisitions[a.label][k] += n 
            

In [12]:
acquisitions

{'bbl1_faceB0_178': {'task-face_run-02_bold': 128},
 'localizer_32channel': {},
 'MPRAGE_TI1100_ipat2': {'T1w': 116},
 'bbl1_cardA0_178': {'task-card_run-01_bold': 124},
 'MoCoSeries': {'asl_moco': 128},
 'bbl1_faceA0_178': {'task-face_run-01_bold': 123},
 'epi_singlerep': {},
 'ep2d_se_pcasl_PHC_1200ms': {'asl': 128},
 'bbl1_cardB0_178': {'task-card_run-02_bold': 124},
 'ep2d_bold_MGH': {'task-rest_bold': 113},
 'B0map_onesizefitsall_v2': {'phase2': 119,
  'phase1': 119,
  'magnitude2': 118,
  'magnitude1': 118},
 '<MPR Collection>': {'T1w': 3},
 'DIFFUSION': {'dwi': 3},
 'DIFFUSION_LOWB': {'dwi': 3},
 'DIFFUSION_TRACEW': {'dwi': 3},
 'T1_MEMPRAGE': {},
 'T1_MEMPRAGE RMS': {'T1w': 5},
 'T1_MEMPRAGE_P': {},
 'DIFFUSION_FA': {'dwi': 3},
 'DIFFUSION_ADC': {'dwi': 3},
 'DIFFUSION_EXP': {'dwi': 3},
 'Space_T2': {'T2w': 3},
 'MPRAGE_TI1100': {'T1w': 6},
 'localizer': {},
 'B0map': {'phase1': 5},
 'bbl1_resting_bold_cap': {'task-rest_bold': 2}}

In [13]:
import pandas as pd

In [14]:
rows = []

for k,v in acquisitions.items():
    
    if v:
        
        sequence = k
    
        for x, y in v.items():
            
            rows.append(pd.DataFrame({'sequence': [k] ,'BIDS_name': [x], 'count': [y]}))

            #print(pd.DataFrame.from_dict(v, orient='index', columns=['BIDS_name', 'count']))


In [15]:
day2_data = pd.concat(rows).reset_index(drop=True)
day2_data

,sequence,BIDS_name,count
0,bbl1_faceB0_178,task-face_run-02_bold,128
1,MPRAGE_TI1100_ipat2,T1w,116
2,bbl1_cardA0_178,task-card_run-01_bold,124
3,MoCoSeries,asl_moco,128
4,bbl1_faceA0_178,task-face_run-01_bold,123
5,ep2d_se_pcasl_PHC_1200ms,asl,128
6,bbl1_cardB0_178,task-card_run-02_bold,124
7,ep2d_bold_MGH,task-rest_bold,113
8,B0map_onesizefitsall_v2,phase2,119
9,B0map_onesizefitsall_v2,phase1,119


In [16]:
day2_data.groupby('count').size() / day2_data['count'].count()*100

count
2       4.166667
3      33.333333
5       8.333333
6       4.166667
113     4.166667
116     4.166667
118     8.333333
119     8.333333
123     4.166667
124     8.333333
128    12.500000
dtype: float64

Certainly for `day2`, less than 5% of the data collected was resting state. Let's check `fndm`:

In [17]:
fndm = [x for x in sessions if 'fndm' in x.label.lower()]

In [18]:
def process_acq(acq):
    
    files = acq.files
    
    names = {}
    
    for f in files:
        
        if f.type != 'nifti':
            
            continue

        bids_filename = get_nested(f, 'info', 'BIDS', 'Filename')

        if bids_filename is None:
            continue
        
        fndm_start = bids_filename.find('fndm')
        end_ses = bids_filename[fndm_start:].find("_")
        bids_filename = bids_filename[fndm_start+end_ses+1:].replace(".nii.gz", "")
        
        if bids_filename not in names.keys():
            names[bids_filename] = 1
        else:
            names[bids_filename] += 1
        
    return names

In [19]:
acquisitions = {}

for d in fndm:
    acqs = d.acquisitions()

    for a in acqs:
                
        
        if a.label not in acquisitions.keys():
            
            names = process_acq(a)
            
            acquisitions[a.label] = names
            
        else:
            
            names = process_acq(a)

            for k, n in names.items():
                
                if k not in acquisitions[a.label].keys():
                    
                    acquisitions[a.label][k] = n 
                    
                else:
                    
                    acquisitions[a.label][k] += n 
            

In [20]:
rows = []

for k,v in acquisitions.items():
    
    if v:
        
        sequence = k
    
        for x, y in v.items():
            
            rows.append(pd.DataFrame({'sequence': [k] ,'BIDS_name': [x], 'count': [y]}))

In [21]:
fndm_data = pd.concat(rows).reset_index(drop=True)
fndm_data

,sequence,BIDS_name,count
0,bbl1_faceA0_178,task-face_run-01_bold,56
1,bbl1_cardB0_178,task-card_run-02_bold,56
2,MPRAGE_TI1100_ipat2,T1w,159
3,bbl1_faceB0_178,task-face_run-02_bold,57
4,ep2d_bold_MGH,task-rest_bold,56
5,B0map_onesizefitsall_v2,phase2,55
6,B0map_onesizefitsall_v2,phase1,55
7,B0map_onesizefitsall_v2,magnitude2,55
8,B0map_onesizefitsall_v2,magnitude1,55
9,bbl1_cardA0_178,task-card_run-01_bold,57


In [22]:
fndm_data.BIDS_name.value_counts() / len(fndm_data.BIDS_name) * 100

task-rest_bold           12.500000
T1w                       8.333333
magnitude1                8.333333
phase1                    8.333333
magnitude2                8.333333
phase2                    8.333333
task-itc_run-02_bold      4.166667
task-itc_run-04_bold      4.166667
task-itc_sbref            4.166667
task-face_run-01_bold     4.166667
task-card_run-02_bold     4.166667
asl                       4.166667
task-itc_run-03_bold      4.166667
asl_moco                  4.166667
task-card_run-01_bold     4.166667
task-face_run-02_bold     4.166667
task-itc_run-01_bold      4.166667
Name: BIDS_name, dtype: float64

Indeed only 5% of the data is resting state bold here.

> Also for "NEFF2" it says there are 4 effort runs, this is not correct... for NEFF (1) there are indeed 4 effort runs that correspond exactly (in timing/task structure but with effort rather than delay) to ITC1-4 in FNDM2, but for NEFF2 there are 2 main effort runs (with a completely different structure from NEFF/ITC) and then a pilot 3rd effort run that was playing around with various things (temporal resolution, spatial resolution, gain vs. loss framing).

Let's see the different NEFF studies:

In [23]:
set([x.label for x in sessions])

{'day2',
 'day2x2',
 'fndm1',
 'fndm2',
 'neff',
 'neff2',
 'neff2x2',
 'neffx2',
 'nodra'}